In [ ]:
%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell

from data_loader import get_pdf_filepaths, load_page_and_line_indexes, load_split_data, load_raw_indexes_list
from pdf_reader import parse_pdf
from data_cleaner import clean_initial_indexes, add_split_data
from data_saver import save_page_and_line_indexes, save_split_data, save_raw_indexes_list
from data_transformer import (
  get_candidates_and_frequencies, 
  add_frequencies_column, 
  add_is_in_toc, 
  add_importance,
  add_position_in_context,
  add_is_named_entity,
  add_length_of_word,
  add_is_named_author,
  add_tfidf,
  get_raw_indexes_list,
  add_is_in_index,
  aggregate_by_candidate
)


InteractiveShell.ast_node_interactivity = "all"

# Configuration vars

In [ ]:
# Configuration vars
DATA_DIR_PATH = "../data/"
PDF_SOURCE_DIR_PATH = DATA_DIR_PATH + "pdf/"
PROCESSED_DATA_DIR_PATH = DATA_DIR_PATH + "processed/"

# Load data

In [ ]:
file_paths = get_pdf_filepaths(PDF_SOURCE_DIR_PATH)
file_paths

# Raw line and page data


In [ ]:

# TODO: Does not work with file_paths[0] and file_paths[2]
for file_path in file_paths[3:4]:
  raw_line_and_page_indexes = parse_pdf(file_path)

  line_and_page_indexes = clean_initial_indexes(raw_line_and_page_indexes)

  # line_and_page_indexes['file_name']
  # line_and_page_indexes['by_line'].head()
  # line_and_page_indexes['by_page'].head()

  save_page_and_line_indexes(
    processed_data_dir_path=PROCESSED_DATA_DIR_PATH, 
    line_and_page_indexes=line_and_page_indexes
  )


# Data set split


In [ ]:
for file_path in file_paths[3:4]:
  line_and_page_indexes = load_page_and_line_indexes(
    processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
    pdf_filepath=file_path
  )
  with_split_data = add_split_data(
    file_path=file_path,
    line_and_page_indexes=line_and_page_indexes
  )

  # with_split_data['by_page_body'].head()
  
  save_split_data(
    processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
    split_data=with_split_data  
  )
 


# Prepare input data frames

In [ ]:
file_path = file_paths[3]

split_data = load_split_data(
    processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
    pdf_filepath=file_path
  )
# split_data['by_line_toc'].head()

(candidates_df, freq_ngrams) = get_candidates_and_frequencies(split_data)

# for file_path in file_paths[3:4]:
#   split_data = load_split_data(
#     processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
#     pdf_filepath=file_path
#   )
#   split_data['by_line_toc'].head()

#   (candidates_df, freq_ngrams) = get_candidates_and_frequencies(split_data)

#   candidates_df.head()

#   with_frequencies = add_frequencies_column(
#     by_pages_body_df=split_data['by_page_body'],
#     candidates_df=candidates_df,
#     freq_ngrams=freq_ngrams
#   )

#   with_frequencies.head()

  

In [ ]:
# candidates_df[candidates_df['candidate_keyword'] == 'socio-cultural']
candidates_df[candidates_df['candidate_keyword'] == 'socio']

In [ ]:
with_frequencies = add_frequencies_column(
  by_pages_body_df=split_data['by_page_body'],
  candidates_df=candidates_df,
  freq_ngrams=freq_ngrams
)

In [ ]:
# with_frequencies[500:510]
# len(freq_ngrams)

In [ ]:
with_is_in_toc = add_is_in_toc(
  candidates_df=with_frequencies, 
  by_line_toc=split_data['by_line_toc']
)

with_is_in_toc.head()


In [ ]:
with_importance = add_importance(candidates_df=with_is_in_toc)

with_importance.head()

In [ ]:
with_position_in_context = add_position_in_context(candidates_df=with_importance)



In [ ]:
with_position_in_context[500:505]

In [ ]:
with_is_named_entity = add_is_named_entity(
  candidates_df=candidates_df,
  df_pages_body=split_data['by_page_body']
)

In [ ]:
with_is_named_entity.head()

In [ ]:
with_length_or_word = add_length_of_word(candidates_df)

In [ ]:
# with_length_or_word[500:503]

In [ ]:
with_is_named_author = add_is_named_author(
  candidates_df=with_length_or_word,
  df_cann_pages_biblio=split_data['by_page_biblio']
)

In [ ]:
with_is_named_author.head()

In [ ]:
with_tfidf = add_tfidf(
  candidates_df=with_is_named_author,
  df_cann_pages_body=split_data['by_page_body']
)

In [ ]:
with_tfidf.head()

In [ ]:
(raw_indexes_list, updated_by_line_index) = get_raw_indexes_list(
  df_cann_lines_index=split_data['by_line_index']
)

In [ ]:
raw_indexes_list

In [ ]:
save_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path,
  raw_indexes_list=raw_indexes_list
)

In [ ]:
clean_indexes = load_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path
)

In [ ]:
# clean_indexes

In [ ]:
with_is_in_index = add_is_in_index(
  candidates_df=candidates_df,
  indexes_list=clean_indexes
)

In [ ]:
with_is_in_index.head()

In [ ]:
aggregate_by_candidate(
  candidates_df=with_is_in_index  
)

In [ ]:
# candidates_df[candidates_df['candidate_keyword'] == 'socio-cultural']
candidates_df[candidates_df['candidate_keyword'] == 'socio']

In [ ]:
with_frequencies = add_frequencies_column(
  by_pages_body_df=split_data['by_page_body'],
  candidates_df=candidates_df,
  freq_ngrams=freq_ngrams
)

In [ ]:
# with_frequencies[500:510]
# len(freq_ngrams)

In [ ]:
with_is_in_toc = add_is_in_toc(
  candidates_df=with_frequencies, 
  by_line_toc=split_data['by_line_toc']
)

with_is_in_toc.head()


In [ ]:
with_importance = add_importance(candidates_df=with_is_in_toc)

with_importance.head()

In [ ]:
with_position_in_context = add_position_in_context(candidates_df=with_importance)



In [ ]:
with_position_in_context[500:505]

In [ ]:
with_is_named_entity = add_is_named_entity(
  candidates_df=candidates_df,
  df_pages_body=split_data['by_page_body']
)

In [ ]:
with_is_named_entity.head()

In [ ]:
with_length_or_word = add_length_of_word(candidates_df)

In [ ]:
# with_length_or_word[500:503]

In [ ]:
with_is_named_author = add_is_named_author(
  candidates_df=with_length_or_word,
  df_cann_pages_biblio=split_data['by_page_biblio']
)

In [ ]:
with_is_named_author.head()

In [ ]:
with_tfidf = add_tfidf(
  candidates_df=with_is_named_author,
  df_cann_pages_body=split_data['by_page_body']
)

In [ ]:
with_tfidf.head()

In [ ]:
(raw_indexes_list, updated_by_line_index) = get_raw_indexes_list(
  df_cann_lines_index=split_data['by_line_index']
)

In [ ]:
raw_indexes_list

In [ ]:
save_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path,
  raw_indexes_list=raw_indexes_list
)

In [ ]:
clean_indexes = load_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path
)

In [ ]:
# clean_indexes

In [ ]:
with_is_in_index = add_is_in_index(
  candidates_df=candidates_df,
  indexes_list=clean_indexes
)

In [ ]:
with_is_in_index.head()

In [ ]:
aggregate_by_candidate(
  candidates_df=with_is_in_index  
)